In [ ]:
import os
import shutil
from glob import glob

import random
import pandas as pd

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

# tf input pipline

In [ ]:
train_path = "./dataset/main_directory/"
test_path = "./dataset/test/"

INPUT_SHAPE = (224,224,3)
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
def parse_image(filename):
    img = tf.io.read_file(filename)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img,INPUT_SHAPE[:2])
    return img

def configure_for_performance(ds):
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.repeat()
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [ ]:
classes = os.listdir(train_path)
num_classes = len(classes)
filenames = glob(train_path+'*/*')
num_images = len(filenames)
print(filenames[0].split(os.sep)[-2])
random.shuffle(filenames)
labels = [classes.index(name.split(os.sep)[-2]) for name in filenames]

filenames_ds = tf.data.Dataset.from_tensor_slices(filenames)
images_ds = filenames_ds.map(
    parse_image,
    num_parallel_calls=AUTOTUNE
)
labels_ds = tf.data.Dataset.from_tensor_slices(labels)

ds = tf.data.Dataset.zip((images_ds, labels_ds))
ds = configure_for_performance(ds)

In [ ]:
for img, label in ds.take(1):
    print(img.numpy()[0].shape)
    img = img.numpy()[0].astype(np.uint8)
    print(img.min(), img.max())
    plt.imshow(img), plt.axis('off')
    plt.show()
    
    print(label.numpy()[0])
    idx = (label.numpy()[0])
    print(classes[idx])

# data augmentation layer
## evaluate() 또는 predict() 호출 시에는 자동으로 비활성화 된다.

In [ ]:
data_augmentation = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    keras.layers.experimental.preprocessing.RandomRotation(0.2),
    keras.layers.experimental.preprocessing.RandomTranslation(0.1,0.1),
])

# model

In [ ]:
transfer_model = keras.applications.VGG16(
    input_shape= INPUT_SHAPE,
    include_top= False,
    weights= 'imagenet',
)
transfer_model.trainable = False
transfer_model.summary()

In [ ]:
finetune_model = keras.Sequential([
    keras.Input(shape=INPUT_SHAPE),
    data_augmentation,
    keras.layers.experimental.preprocessing.Rescaling(1./255),
    transfer_model,
    keras.layers.Flatten(),
    keras.layers.Dense(num_classes, activation='softmax')
],"VGG16")
finetune_model.summary()

finetune_model.compile(loss='sparse_categorical_crossentropy',
                      optimizer=keras.optimizers.Adam(learning_rate=0.0002),
                       metrics='accuracy')                    

In [ ]:
step = num_images//BATCH_SIZE
tf.random.set_seed(3)

hist = finetune_model.fit(
    ds, epochs=2, steps_per_epoch=step
)

In [ ]:
for i,j in ds.take(2):
    print(j.numpy())